Puesto que la funcionalidad de añadir un script no funciona en este momento, no hemos podido incluir el script de utilidades de la manera vista en clase, así que lo hemos puesto todo en una clase con métodos estáticos en esta primera celda.

In [ ]:
# =============================================================================
# Imports
# =============================================================================

# Third party
import cufflinks as cf
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.utils.validation import check_is_fitted
from imblearn.base import BaseSampler
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.validation import check_is_fitted
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import OneHotEncoder

# =============================================================================
# Initialization
# =============================================================================

# Set the offline display mode
cf.go_offline(connected="True")


# =============================================================================
# Functions
# =============================================================================
class utils:
    @staticmethod
    def _filter_numerical_data(data):
        """Filter the numerical data."""
        return data.select_dtypes(include="number")

    @staticmethod
    def _filter_categorical_data(data):
        """Filter the categorical data."""
        return data.select_dtypes(include="category")

    @staticmethod
    def _plot_barplot(data, var):
        """Plot univariate distribution of the categorical variable."""
        # Count the relative frequency of unique values
        count = data[var].value_counts(normalize=True)

        return count.iplot(kind="bar")

    @staticmethod
    def load_data(filepath, index, target):
        """Load and return the dataset."""
        data = pd.read_csv(filepath, index_col=index)

        # Convert the target variable to categorical
        data[target] = data[target].astype("category")

        return data

    @staticmethod
    def divide_dataset(data, target):
        """Divide the dataset into data and target."""
        return (data.drop(target, axis=1), data[target])

    @staticmethod
    def join_dataset(X, y):
        """Join the data and target into dataset."""
        data = X.copy(deep=True)

        target = y.name
        data[target] = y

        return data

    @staticmethod
    def plot_histogram(data):
        """Plot univariate distribution of the numerical data."""
        numerical_data = utils._filter_numerical_data(data)

        return numerical_data.iplot(kind="hist")

    @staticmethod
    def plot_barplot(data):
        """Plot univariate distribution of the categorical data."""
        categorical_data = utils._filter_categorical_data(data)

        # Add a dropdown widget to select
        # the categorical feature to plot
        var = categorical_data.columns
        data = widgets.fixed(categorical_data)

        widgets.interact(utils._plot_barplot, data=data, var=var)

    @staticmethod
    def create_widget(function,**kwargs):
        """Create custom widget."""
        widgets.interact(function, **kwargs)

    @staticmethod
    def plot_pairplot(data, target):
        """Plot a multivariate distribution of the data."""
        # Drop the target variable from the plot
        dimensions = data.drop(target, axis=1)

        return px.scatter_matrix(data,
                                 dimensions=dimensions,
                                 color=target)

    @staticmethod
    def _plot_boxplot(data,variable_shown):
        """Plot a boxplot for a single variable."""
        numerical_data = utils._filter_numerical_data(data)
        return px.box(numerical_data,y=variable_shown, points="all")

    @staticmethod
    def plot_boxplot(data):
        """Plot a boxplot for multiple variables."""
        numeric_data = utils._filter_numerical_data(data)  
        var = numeric_data.columns
        data = widgets.fixed(numeric_data)
        return widgets.interact(utils._plot_boxplot, data=data,variable_shown=var)

    @staticmethod
    def plot_scatterplot(data,variable_shown):
        """Plot a boxplot for a single variable."""
        numerical_data = utils._filter_numerical_data(data)
        return px.scatter(numerical_data, y=variable_shown)

    @staticmethod
    def plot_correlationmatrix(data):    
        """Plot a correlation matrix for a pandas dataframe."""
        numerical_data = utils._filter_numerical_data(data)
        return px.imshow(data.corr())
    @staticmethod
    def plot_conditional_barplot(data,target,normalize=False,marginal=None):
        """Plot a conditional barplot"""
        if normalize:
            data_numeric = utils._filter_numerical_data(data)
            data_numeric_normalized = (data_numeric-data_numeric.mean())/data_numeric.std()
            data = pd.concat([data_numeric_normalized, utils._filter_categorical_data(data)],axis=1)

        return px.histogram(data, barmode="group", facet_col=target,marginal=marginal)



    @staticmethod
    def plot_roc_curve(model,
                        X_train,y_train,
                        X_test, y_test,
                        pos_label=None,
                        ):
        """Plot roc curve and area under it"""
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        if pos_label:
            y_pred = np.where(y_pred==pos_label,1,0)
            y_test = np.where(y_test==pos_label,1,0)
            
        fpr, tpr, thresholds = roc_curve( y_test,y_pred, pos_label=1)
        fig = px.area(
            x=fpr, y=tpr,
            title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
            labels=dict(x='False Positive Rate', y='True Positive Rate'),
            width=700, height=500
        )
        fig.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
        )

        fig.update_yaxes(scaleanchor="x", scaleratio=1)
        fig.update_xaxes(constrain='domain')
        return fig
    # =============================================================================
    # Evaluation
    # =============================================================================
    @staticmethod
    def evaluate(model,
                 X_train, X_test,
                 y_train, y_test):
        """Evaluate the model by holdout."""
        # Fit the model to the training data
        clf = model.fit(X_train, y_train)

        # Predict the target classes
        y_pred = clf.predict(X_test)

        # Compute the fraction of correctly classified samples
        accuracy = accuracy_score(y_test, y_pred)

        # Plot the confusion matrix
        disp = plot_confusion_matrix(clf, X_test, y_test)

        # Add the accuracy to the plot title (up to five decimals)
        disp.ax_.set_title(f"Accuracy = {accuracy:.5f}")


    @staticmethod
    def evalute_classification_report(model,
                                    X_train, X_test,
                                    y_train, y_test):
        """Evaluate the model by holdout with details"""
         # Fit the model to the training data
        clf = model.fit(X_train, y_train)

        # Predict the target classes
        y_pred = clf.predict(X_test)

        return classification_report(y_test,y_pred)
    
    
    # =============================================================================
    # Auxiliary functions
    # =============================================================================

    @staticmethod
    def count_outliers(data, columns):
        '''Count number of outliers in given columns'''
        indexes = set()
        for column in columns:
            q25, q75 = np.nanpercentile(data[column], [25, 75])
            iqr = q75-q25
            indexes.update(data.loc[((data[column] < q25-1.5*iqr) | (data[column] > q75+1.5*iqr))].index)
        return len(indexes),data.loc[indexes]

    @staticmethod
    def round_values_analysis(data,target,variable,decimal=2):
        ''' Evaluate the amount of noise introduced by rounding a variable'''
        print(f"Número de valores únicos sin redondeo: {data[variable].nunique()}" )
        print(f"Número de valores únicos con redondeo: {data[variable].round(decimals=decimal).nunique()}")

        df = data[[variable,target]].groupby(variable).apply(lambda grp: grp.shape[0]-grp[target].value_counts()[0]).reset_index(name='Count')
        print(f"Número de casos para cada valor único de {variable} distintos de la clase mayoritaria (Sin redondeo): {df['Count'].sum()}")

        df = data.copy(deep=True)
        df[variable] = df[variable].round(decimal);
        df = df[[target,variable]].groupby(variable).apply(lambda grp: grp.shape[0]-grp[target].value_counts()[0]).reset_index(name='Count')
        print(f"Número de casos para cada valor único de {variable} distintos de la clase mayoritaria (Con redondeo) {df['Count'].sum()}")


    # =============================================================================
    # Transformers
    # =============================================================================    
    '''
    Transformador para la eliminación de columnas en el pipeline
    '''    
    @staticmethod
    class ColumnRemoverTransformer(BaseEstimator,TransformerMixin):
        def __init__(self,columns_to_drop):
            self.columns_to_drop=columns_to_drop

        def fit(self,X,y=None):
            return self

        def transform(self,X,y=None):
            X = X.copy()
            return X.drop(self.columns_to_drop,axis=1)


    '''
    Eliminación de outliers en el pipeline de imblearn mediante la fórmula estadística
    '''
    @staticmethod
    class OutliersRemovalTransformer(BaseSampler):
        def __init__(self,columns):
            self.columns = columns

        def fit_resample(self, X, y):
            X = X.copy()
            y = y.copy()
            for column in self.columns:
                q75,q25 = np.nanpercentile(X[column], [75, 25])
                iqr = q75-q25
                outliers_indices = X[(X[column]>(1.5*iqr+q75)) | (X[column]<(q25-1.5*iqr))].index
                X.drop(outliers_indices,axis=0,inplace=True)
                y.drop(outliers_indices,axis=0,inplace=True)
            return X, y

        def _fit_resample(self, X, y):
            return X, y

    '''
    Sustitución de valores en las variables predictoras (utilizado para remplazar valores pérdidos por el estandard np.nan)
    '''
    @staticmethod
    class Replace(BaseEstimator,TransformerMixin):
        def __init__(self, columns_to_modify, value_to_replace, fill_value):
            self.columns_to_modify = columns_to_modify
            self.value_to_replace = value_to_replace
            self.fill_value = fill_value

        def fit(self, X, y=None):
            return self

        def transform(self, X, y=None):
            X = X.copy()
            X[self.columns_to_modify]=X[self.columns_to_modify].replace({self.value_to_replace: self.fill_value})
            return X


    '''
    Imputador de valores númericos mediante un árbol de regresión
    '''
    @staticmethod
    class DecisionTreeImputer(BaseEstimator,TransformerMixin):
        def __init__(self,target,columns,seed=None):
            '''
            inicializa el imputador:
            :param target: columna que queremos imputar
            :param seed: semilla que se utiliza para el árbol de regresión
            :param columns: columnas disponibles (incluye target)
            :return:
            '''
            self.tree=DecisionTreeRegressor(criterion="mse",random_state=seed)
            self.target=target
            self.seed=seed
            self.columns = columns

        def fit(self,X,y=None):
            '''
            entrena el transformador:
            Basándonos en los datos no perdidos predecimos utilizando el resto de variables
            :param X: atributos predictores
            :param y: variable no utilizada por el Pipeline
            :return:
            '''
            X=X.copy()
            filtered = X.loc[~(X[self.columns].isnull().any(axis=1)), self.columns]
            filtered_without_target = filtered.drop(self.target, axis=1)
            self.tree.fit(filtered_without_target,filtered[self.target])
            return self

        def transform(self,X,y=None):
            '''
            transforma los datos:
            :param X: atributos predictores, donde se encuentra la columna que queremos imputar
            :param y: variable no utilizada por el Pipeline
            :return: devuelve el resultado sin valores perdidos para la variable objetivo del imputador
            '''
            check_is_fitted(self.tree)
            X=X.copy()
            X.loc[X[self.target].isnull(), self.target] = self.tree.predict(np.array(X.loc[X[self.target].isnull(), self.columns].drop(self.target, axis=1)))
            return X

    '''
    Imputador por la media de valores perdidos para un grupo de columnas en particular. Espera un DataFrame y devuelve un DataFrame.
    '''
    @staticmethod   
    class CustomSimpleImputer(BaseEstimator,TransformerMixin):
        def __init__(self, target):
            self.target = target
            self.imputer = SimpleImputer()

        def fit(self, X, y=None):
            self.imputer.fit(X[self.target])
            return self

        def transform(self, X, y=None):
            check_is_fitted(self.imputer)
            X = X.copy()
            X[self.target] = self.imputer.transform(X[self.target])
            return X


    '''
    Discretizador por la media, utiliza en la salida encoding OneHotEncoder donde hemos incluido el nombre de las columnas manualmente.
    '''
    @staticmethod
    class MeanDiscretizer(BaseEstimator, TransformerMixin):
        def __init__(self,columns_to_discretize):
            self.columns_to_discretize=columns_to_discretize
            self.means = []
            self.discretizer = OneHotEncoder()

        def fit(self,X,y=None):
            X = X.copy()
            for column in self.columns_to_discretize:
                self.means.append(X[column].mean())
                X[column] = np.where(X[column] < X[column].mean(),0,1)
            self.discretizer.fit(X[self.columns_to_discretize],y)
            return self

        def transform(self,X,y=None):
            X = X.copy()
            check_is_fitted(self.discretizer)
            for mean,column in zip(self.means,self.columns_to_discretize):
                X[column] = np.where(X[column] < mean,0,1)
            nuevas_columnas = []
            for column in self.columns_to_discretize:
              nuevas_columnas.append(f"{column}_menor")
              nuevas_columnas.append(f"{column}_mayor")
            X[nuevas_columnas] = self.discretizer.transform(X[self.columns_to_discretize]).toarray()
            X=X.drop(self.columns_to_discretize,axis=1)
            return X

# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

### Grupo L:

* David Mora Garrido
* Pablo Luque Romero

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo



En esta práctica vamos a trabajar algunos de los aspectos más importantes del proceso *KDD* (*Knowledge Discovery from Data*):

* Almacenamiento y carga de datos
* Análisis exploratorio de datos
* Preprocesamiento de datos
* Validación de modelos de clasificación

Para ello, aprenderemos a manipular y visualizar los datos mediante distintas funciones de las librerías `pandas` y `plotly`. Además, aprenderemos a utilizar algoritmos de clasificación como *Zero-R* y árboles de decisión usando la librería `scikit-learn`.

El objetivo de la práctica será aprender a cargar, explorar y preparar nuestros datos, aprender y validar distintos modelos de clasificación y ser capaces de interpretar los resultados obtenidos. Para lograrlo, utilizaremos dos conjuntos de datos sintéticos:

- `pima_diabetes`: https://www.kaggle.com/uciml/pima-indians-diabetes-database
- `wisconsin`: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

La descripción de cada uno de estos conjuntos de datos se encuentra en el enlace correspondiente.

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import make_pipeline as imblearn_make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.dummy import DummyClassifier
# import md_L_practica1_utils as utils 

Establecemos un seed que utilizaremos para toda la libreta, de manera que los experimentos sean reproducibles.

In [ ]:
seed = 27913

# Conjunto de datos I: Diabetes

En primer lugar cargaremos los datos y crearemos una partición en conjunto de *test* y de entrenamiento, mediante la técnica de *holdout* estratificado. Aunque lo ideal sería utilizar una técnica de validación cruzada subdividiendo el conjunto de entrenamiento en carpetas para evitar que la aleatoriedad pueda jugar a favor o en contra de los resultados obtenidos, no es el objetivo de la práctica y por lo tanto no utilizaremos este método.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = None
target = "Outcome"

data_pima_indians = utils.load_data(filepath, index, target)
(X, y) = utils.divide_dataset(data_pima_indians, target=target)

Dividimos los datos:

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)



Por comodidad a la hora de representar los datos, sustituiremos los estados de la variable clase *Outcome* por sus correspondientes categorías nominales

In [ ]:
y_train=y_train.replace({0: "ND", 1: "D"}).astype("category")
y_test=y_test.replace({0: "ND", 1: "D"}).astype("category")

Unimos lo datos de training:

In [ ]:
data_train = utils.join_dataset(X_train,y_train)

Realizamos un muestro aleatorio de nuestros datos para comprobar que la carga se ha realizado correctamente. Tanto el muestreo, como todos los análisis que realizaremos a partir de aquí serán sobre el conjunto de entrenamiento (con el fin de evitar un *data leak*).

In [ ]:
data_train.sample(5,random_state=seed)

### 1. Análisis exploratorio de datos mediante gráficas y estadísticos.

Vamos a obtener una primera visión global descriptiva de los datos para evaluar rangos, medias y dispersión en los atributos. También imprimiremos un resumen de la variable clase.

In [ ]:
display(data_train.describe(include="number"))
display(data_train.describe(include="category"))

Nuestro conjunto de entrenamiento representa el 70% de los datos totales y en total está compuesto por 573 registros. Vemos que el atributo *Pregnancies* tiene un coeficiente de variación $\dfrac{\sigma}{\overline{x}}$  elevado (cercano a uno), por lo tanto hay una dispersión importante y lo mismo ocurre con _Skinthickness_. En cambio la variable glucosa parece tener los datos más concentrados en torno a la media (en terminos de valor relativo).

Con respecto a la variable clase, observamos que hay dos clases, como indicaba la descripción del conjunto de datos en *Kaggle* y que la clase mayoritaria es ND (0). Sin embargo, esto no indica que la base de datos no esté balanceada así que a continuación veremos la distribución de la variable clase.

In [ ]:
utils.plot_barplot(data_train)

El conjunto de datos no está balanceado. Como es habitual en la población, hay más individuos que no padecen de diabetes que diabéticos. Este reparto no balanceado se ha mantenido en los datos. Donde tenemos:
* No diabéticos: 65%
* Diabéticos: 35%

In [ ]:
utils.plot_histogram(data_train)

A partir de este histograma podemos estudiar la distribución de valores de cada variable predictora, y la presencia o no de valores anormales que podrían ser ruido:

* `Pregnancies`: por razones obvias, esta variable parece seguir una distribución sesgada hacia valores pequeños (la media está entre 3 y 4) y decreciente en casi todo su dominio observado, pareciéndose a una distribución geométrica. Además, parece haber outliers; con los datos que podemos ver en la visión descriptiva, consideraríamos outliers a aquellas instancias con Pregnancies >= 14. Sin embargo, al ser pocos casos, y considerando que posiblemente no sean ruido por la propia naturaleza de la variable, no sería primordial eliminarlos.

* `Glucose`: esta variable sigue claramente una distribución normal. Buscando información sobre niveles de glucosa en sangre (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1838950/, https://www.ncbi.nlm.nih.gov/books/NBK430900/) y con los valores , podemos ver cómo hay una claro problema de casos con hiperglucemia (valores más alla de 125), teniendo en cuenta que la media en personas sin diabetes es de unos 90 mg/L, mientras que la media en este caso ronda los 120 mg/L (e incluso más, porque, como veremos, hay valores desconocidos que generan ruido). Otro signo que indica esto es el hecho de que el primer cuartil (99 mg/L) también es mayor que 90 mg/L. Por último, hay que considerar que es imposible tener un nivel de glucosa en sangre de 0 mg/L, porque supondría muerte cerebral (con niveles algo mayores de 0 también), por tanto estos casos son valores desconocidos que tendremos que imputar.

* `BloodPressure`: esta variable también parece seguir una distribución normal. Si buscamos información (por ejemplo, https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings), podemos ver que casi todos los valores entran dentro de lo que se podría observar (teniendo en cuenta que por el dominio observado nos referimos a la presión diastólica), aunque, por supuesto, ciertos casos implicarían hipertensión o hipotensión; en este último caso, aunque existan valores muy bajos (entre 20 y 40), sería interesante mantenerlos, puesto que pueden ser consecuencia de tener diabetes (https://medlineplus.gov/ency/article/007278.htm). Por último, también existen valores desconocidos (24 casos) que tendremos que imputar, puesto que es imposible tener una presión sanguínea de 0.

* `SkinThickness`: a pesar de la gran cantidad de valores desconocidos, la distribución de los conocidos parece ser normal. Esta medida se toma en el triceps; si buscamos información sobre este método y los valores que se suelen obtener, podemos ver que los valores de muchas de las instancias este *dataset* son mayores de lo normal. Sin embargo, hay que tener en cuenta que todos los casos son de mujeres, que normalmente tienen una medida mayor que los hombres, e incluso el ratio con el que aumenta por la edad es mayor también en mujeres que en hombres (como se menciona en el *abstract* y en la tercera tabla del siguiente artículo: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1619658/pdf/amjph00685-0050.pdf). Por último, dada la gran cantidad de valores desconocidos en esta variable, tras el estudio de esta gráfica la estudiaremos con más detalle para determinar si vale la pena mantenerla.

* `Insulin`: al igual que `SkinThickness`, esta variable tiene muchos valores desconocidos, e incluso más que la citada. En principio no parece seguir una distribución normal, aunque en general no podemos hacer muchas suposiciones debido al número de valores desconocidos. Por otro lado, las mediciones de esta variable, según este artículo https://www.sciencedirect.com/science/article/pii/S2352914816300016, están expresadas en μU/mL, y fueron tomados en periodos de 2 horas tras la ingesta de glucosa; sin embargo, según este artículo https://emedicine.medscape.com/article/2089224-overview (si no se puede ver/registrarse, se puede darle al símbolo de imprimir para ver la Tabla 1), los valores normales están en el rango 16-166 (que podría coincidir con el orden de magnitud de nuestras medidas), pero en este caso se indica que son mIU/L (quizás se refiere a la misma unidad pero no está del todo claro, si suponemos que es mili U/L, es equivalente a micro U/mL; en general no hay mucha información sobre este tipo de medición y los rangos sin ambigüedades, especialmente para la medición pasadas 2 horas de la ingesta de glucosa). Además, por la gran cantidad de valores desconocidos, tendremos que determinar si mantenerla.

* `BMI`: las mediciones de esta variable parecen seguir una distribución normal. Lo más remarcable es que la media y la mediana esté en torno a 32, que es un valor que se encuentra en el rango de obesidad (según https://www.cdc.gov/obesity/adult/defining.html), por lo que podríamos pensar que existe un sesgo en nuestra muestra. Sin embargo, también tendríamos que tener en cuenta que pueden existir ciertos valores culturales que asocian un mayor volumen corporal con la fertilidad, prosperidad, etc. (y hay que tener en cuenta que los datos se refieren a una etnicidad no occidental/caucásica). Por último, existen valores desconocidos que imputaremos al no suponer una proporción considerable respecto al total de casos.

* `DiabetesPedigreeFunction`: esta variable no parece seguir una distribución normal o, al menos, que no sea resultado de la combinación de más de una distribución normal. Según este artículo https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2245318/pdf/procascamc00018-0276.pdf, esta función es una síntesis que nos informa sobre el historial de diabetes en familiares cercanos y, por extensión, la influencia de la genética en el hecho de que nuestro caso desarrolle o no diabetes. Por tanto, parece una variable predictora importante, que además no tiene valores desconocidos.

* `Age`: por último, esta variable no sigue una distribución normal (y tampoco es posible que lo haga), cuando realmente es una pirámide de población centrada en el género femenino donde falta la base (< 21 años). Lo único remarcable es que la media y la mediana rondan los 30 años, cuando en Estados Unidos está cerca de los 40, lo que puede indicar que, o bien la muestra está sesgada, o que en esta etnicidad influyen diferentes factores que la hacen más baja (aparte de las sociales/culturales, una de ellas podría ser la propia diabetes). Por eso en este tipo de variables predictoras quizás no nos interese eliminar lo que podría identificarse como outliers ya que pueden aportar información valiosa (y en principio, por el significado de esta variable, en principio no sería lo que conocemos como un valor 'anormal').

Efectivamente, aunque en los datos no aparece con *Nan* dada la semántica del problema no es posible que los valores de `Insulin`, `Glucose`, `BMI`, `SkinThickness` o `BloodPressure` sean igual a 0. En principio podemos trabajar con ellos siempre y cuando no representen más de un 20-25% de la variable.


In [ ]:
data_train[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]].apply(lambda x: f"{(x[x==0].shape[0]/data_train.shape[0])*100:.2f}%")

Observamos que mientras que `Glucose`, `BMI` y `BloodPressure` tienen una proporción de valores perdidos razonable, `SkinThickness` e `Insulin` no. Con casi la mitad de los valores perdidos para `Insulin` y más de un 30% para `SkinThickness` cualquier suposición que podamos hacer sobre la variable sería infundada por lo tanto en el preprocesamiento la eliminaremos y en el resto del análisis no las trataremos.

Sin embargo antes de confirmar que estas variables carecen de significado a nivel estadístico vamos a comprobar si existe alguna relación con la variable clase que nos pueda permitir obtener información adicional del propio hecho de que un valor se haya perdido. Para ello imprimiremos la distribución de la variable clase dadas estas variables.

En primer lugar analizaremos por separado la distribución de la variable clase dada `Insulin`, luego dada `SkinThickness` y por último conjuntamente. Puesto que el uso de esta gráfica es puntual y que no requiere mucho código la crearemos en la siguiente celda

In [ ]:
import ipywidgets as widgets

data_insulin_skinthickness_missing_check = [
    data_train[data_train["Insulin"] == 0],
    data_train[data_train["Insulin"] != 0],
    data_train[data_train["SkinThickness"] == 0],
    data_train[data_train["SkinThickness"] != 0],
    data_train[(data_train["SkinThickness"]==0) & (data_train["Insulin"]==0)],
    data_train[(data_train["SkinThickness"]==0) & (data_train["Insulin"]!=0)],
    data_train[(data_train["SkinThickness"]!=0) & (data_train["Insulin"]==0)],
    data_train[(data_train["SkinThickness"]!=0) & (data_train["Insulin"]!=0)],
   ]

labels = [
          ("Insulin == 0",0),
          ("Insulin != 0",1),
          ("Skinthickness == 0",2),
          ("Skinthickness != 0",3),
          ("Skinthickness == 0 & Insulin == 0 ",4),
          ("Skinthickness == 0 & Insulin != 0 ",5),
          ("Skinthickness != 0 & Insulin == 0 ",6),
          ("Skinthickness != 0 & Insulin != 0 ",7)
]

@widgets.interact
def plot_barplot_for_missing_values(labels=labels):
    count = data_insulin_skinthickness_missing_check[labels][target].value_counts(normalize=True)
    return count.iplot(kind="bar")


Por si sola los valores perdidos de la variable `Insulin` no guardan relación con la variable clase ya que se mantiene la distribución tanto cuando el valor no aparece como cuando sí que lo hace.

Por si sola los valores perdidos de la variable de `SkinThickness` no guardan relación con la variable clase ya que se mantiene la distribución.

Si analizamos como varía la distribución de la variable clase combinado con el hecho de que faltan valores para `Insulin` y/o `SkinThickness` observamos que, aunque el valor fluctúa ligeramente, se mantiene un reparto similar al de la variable clase inicial por lo tanto podemos concluir que son variables cuyos valores perdidos no aportan información y que podemos eliminar en el preprocesamiento.


Para diferenciar claramente los valores perdidos lo sustituiremos por `np.nan`, de manera que no afecte a los cálculos para la detección de outliers.


In [ ]:
data_train[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]]=data_train[["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]].replace({0: np.nan})

Comprobamos que se han reemplazado correctamente:

In [ ]:
data_train[pd.isnull(data_train["Glucose"])]

A continuación visualizaremos mediante un diagrama de cajas y bigotes las variables para las cuales intuimos (por el histograma de las distribuciones) que pueden tener outliers.

In [ ]:
utils._plot_boxplot(data_train,variable_shown="Pregnancies")

En el diagrama vemos que únicamente hay 4 outliers cuya eliminación no afectará a la distribución y ayudará a evitar el sobreajuste en el proceso de clasificación.

In [ ]:
utils._plot_boxplot(data_train,variable_shown="Glucose")

Para la variable `Glucose` no existen *outliers* y puesto que la imputación de valores perdidos, ya sea por la media, _k-medias_ o mediante un árbol de regresión, obtendrá valores basándose en los datos del conjunto es de esperar que los valores imputados estén en la norma de la variable.

In [ ]:
utils._plot_boxplot(data_train,variable_shown="BloodPressure")

En el caso de la variable `BloodPressure`, podemos ver que no hay muchos *outliers*, por lo que tenemos que determinar si mantenerlos (en el caso de que, por ejemplo, formen un grupo con una amplia mayoría para uno de los resultados de la clase) o no (si, por ejemplo, mantienen la misma proporción en los resultados de la clase que nuestra muestra utilizada para el entrenamiento).

En primer lugar, sabemos que son *outliers* aquellos casos cuyo valor para la variable predictora en cuestión esté fuera del siguiente intervalo:

$$[Q1-1.5\cdot IQR,Q3+1.5\cdot IQR]$$


Comenzemos por aquellos valores que podemos considerar menores de lo habitual en nuestra muestra:

In [ ]:
q25,q75 = np.nanpercentile(data_train['BloodPressure'],[25,75])
iqr = q75-q25
data_train[(data_train['BloodPressure'] < q25-1.5*iqr) & (data_train['BloodPressure'] != 0)]

Como podemos ver, en este grupo de casos se mantiene la proporción de valores de la clase de todo el conjunto de entrenamiento. Por tanto, podríamos eliminarlos, ya que no aportan información.

Por otro lado, veamos aquellos casos con un valor mayor de lo habitual en nuestra muestra:

In [ ]:
outliers_superiores = data_train[(data_train['BloodPressure'] > q75+1.5*iqr)]
display(outliers_superiores)
clase_outliers_superiores = outliers_superiores['Outcome'].value_counts()

print("\nProporción de valores de los outliers de la parte superior:")
display(clase_outliers_superiores/clase_outliers_superiores.sum())

En este caso, la muestra nos indica que hay algún caso más de diabetes de lo habitual en la muestra de entrenamiento completa. Sin embargo, para determinar si estos casos podrían ser útiles en el entrenamiento de nuestros modelos, sería interesante ver valores en la frontera, por ejemplo aquellos casos con un valor comprendido en el itervalo $[Q_{75} + 0.5\cdot IQR, Q_{75} + 1.5\cdot IQR]$:

In [ ]:
frontera_superior = data_train[(data_train['BloodPressure'] > q75+0.5*iqr) & (data_train['BloodPressure'] <= q75+1.5*iqr)]
clase_frontera_superior = frontera_superior['Outcome'].value_counts()

display(clase_frontera_superior)
print("\nProporción de valores en la frontera superior:")
display(clase_frontera_superior/clase_frontera_superior.sum())

Como podemos ver, para los casos con un valor de `BloodPressure` mayor que $ Q_{75}+1.5\cdot IQR $, la proporción de casos con diabetes (0.429) es menor que la de los casos comprendidos en la frontera que hemos definido (0.444), pero no dista mucho. Por tanto, realmente eliminar aquellos outliers no afectaría demasiado a esta frontera.


In [ ]:
utils._plot_boxplot(data_train,variable_shown="BMI")

En cuanto a la variable `BMI`, podemos ver que sólo existen outliers en la parte superior del dominio. Para ver si nos interesa mantener o no estos *outliers*, sigamos el procedimiento anterior:

In [ ]:
q25,q75 = np.nanpercentile(data_train['BMI'],[25,75])
iqr = q75-q25

outliers_superiores = data_train[(data_train['BMI'] > q75+1.5*iqr)]
display(outliers_superiores)
clase_outliers_superiores = outliers_superiores['Outcome'].value_counts()

print("\nProporción de valores de los outliers de la parte superior:")
display(clase_outliers_superiores/clase_outliers_superiores.sum())

Como podemos ver, en la muestra que comprende los *outliers* cambia la proporción de valores de la clase respecto al conjunto de entrenamiento completo, ya que un 80% de casos son diabéticos. Sin embargo, vamos a mirar también la frontera, ya que puede haber también una amplia mayoría de casos cuyo valor de clase sea diabético, y por tanto en ese caso no merezca la pena mantener los outliers:

In [ ]:
frontera_superior = data_train[(data_train['BMI'] > q75+iqr) & (data_train['BMI'] <= q75+1.5*iqr)]
clase_frontera_superior = frontera_superior['Outcome'].value_counts()

display(clase_frontera_superior)
print("\nProporción de valores en la frontera superior:")
display(clase_frontera_superior/clase_frontera_superior.sum())

Como podemos ver, en la frontera que definimos hay un 65% de casos que son diabéticos, por lo que hay una diferencia notable con la proporción de diabéticos en la muestra de *outliers*. Dicho esto, tenemos que tener en cuenta también que el número de casos *outliers* es menor que el que hay en esta frontera, por lo que quizás al añadir los *outliers* no cambia demasiado la proporción de diabéticos:


In [ ]:
frontera_superior_y_outliers = pd.concat([outliers_superiores, frontera_superior])
clase_frontera_superior_y_outliers = frontera_superior_y_outliers['Outcome'].value_counts()

print("\nProporción de valores en la frontera superior junto con outliers:")
display(clase_frontera_superior_y_outliers/clase_frontera_superior_y_outliers.sum())

Efectivamente, al añadir los *outliers* a la frontera, la proporción de diabéticos crece ligeramente (de 0.65 a 0.68), pero realmente no supone un cambio significativo; por tanto, y para no sobreajustar, convendría eliminar los outliers de esta variable.

Por último, podemos aventurar a decir que `BMI` puede ser una variable predictora clave, ya que en la muestra de la frontera superior las proporciones iniciales del conjunto de entrenamiento se han invertido, habiendo aproximadamente 2/3 de casos de diabéticos mientras que los no diabéticos suponen 1/3. Este incremento del 30% en los casos de diabéticos es importante.

In [ ]:
utils._plot_boxplot(data_train,variable_shown="DiabetesPedigreeFunction")

In [ ]:
q25,q75 = np.nanpercentile(data_train['DiabetesPedigreeFunction'],[25,75])
iqr = q75-q25
outliers = data_train[(data_train['DiabetesPedigreeFunction'] > q75+1.5*iqr)]
print(f"Número total de outliers para la variable DiabetesPedigreeFunction: {outliers.shape[0]}")

Eliminar los outliers en este caso conllevaría una pérdida de información importante ya que nuestra base de datos no es muy grande. Vamos a comprobar si el comportamiento de la variable clase en estos datos es similar al de la media. Al igual que con las otras variables también incluiremos los valores cercanos a la frontera

In [ ]:
print("Comportamiento en el rango intercuartil:")
print(data_train[(q25<data_train['DiabetesPedigreeFunction']) & (data_train['DiabetesPedigreeFunction'] <q75)]["Outcome"].value_counts(normalize=True))


print("Incluyendo los outliers:")
print(data_train[(data_train['DiabetesPedigreeFunction'] > q75+iqr)]["Outcome"].value_counts(normalize=True))

print("Sin incluir los outliers:")
print(data_train[(data_train['DiabetesPedigreeFunction'] > q75+iqr) & (data_train['DiabetesPedigreeFunction'] < q75+1.5*iqr)]["Outcome"].value_counts(normalize=True))

Recordemos que en este caso estamos intentando diagnosticar diabetes con una base de datos no balanceada por lo tanto cualquier grupo de registros que nos pueda aportar información sobre la variable clase es de vital importancia. Descartar 24 *outliers* sabiendo que la proporción de la variable clase se invierte a favor de los diabéticos para estos datos no es recomendable sabiendo que con ello estaríamos aumentando la desproporción en la variable clase.



Para la variable `DiabetesPedigreeFunction` podría ser interesante reducir la precisión a 2 o 3 decimales para reducir el número de valores que tendrá que probar el algoritmo de clasificación para evaluar la entropía de las particiones.

In [ ]:
utils.round_values_analysis(data_train,target,"DiabetesPedigreeFunction",2)

El número de casos cuyo valor de clase sería distinto del valor de clase mayoritaria tras el redondeo para cada valor único de `DiabetesPedigreeFunction` aumenta al triple que si no hiciéramos el redondeo. Por tanto aunque el número de puntos de corte baje considerablemente, el ruido y la perdida de información que introducimos en la variable no compensa.

In [ ]:
utils._plot_boxplot(data_train,variable_shown="Age")

Al igual que antes, en esta variable predictora sólo exiten outliers por la parte superior de su dominio observado; veamos estos casos en detalle:

In [ ]:
q25,q75 = np.nanpercentile(data_train['Age'],[25,75])
iqr = q75-q25

outliers_superiores = data_train[(data_train['Age'] > q75+1.5*iqr)]
clase_outliers_superiores = outliers_superiores['Outcome'].value_counts()
display(clase_outliers_superiores)

print("\nProporción de valores de los outliers de la parte superior:")
display(clase_outliers_superiores/clase_outliers_superiores.sum())

Podemos ver cómo en los *outliers* de esta variable la proporción de no diabéticos es mayor que la del conjunto de entrenamiento (aprox. 0.77 contra 0.65). Esto podría ser una razón para mantener los *outliers*, pero hay que tener en cuenta que quizás este hecho realmente no aporte información, ya que lo normal es que si se llega a tener una edad mayor, es debido a que probablemente no se tengan enfermedades como la diabetes. Además, dado que la clase no está balanceada y hay más casos de no diabéticos que de diabéticos, un incremento en la proporción de los primeros no es tan significativo como un incremento en la de los segundos. Aún así, miremos como es costumbre en la frontera:

In [ ]:
frontera_superior = data_train[(data_train['Age'] > q75+iqr) & (data_train['Age'] <= q75+1.5*iqr)]
clase_frontera_superior = frontera_superior['Outcome'].value_counts()

display(clase_frontera_superior)
print("\nProporción de valores en la frontera superior:")
display(clase_frontera_superior/clase_frontera_superior.sum())

Como podemos ver, en la frontera la proporción de diabéticos aumenta al 0.44, respecto al 0.35 del conjunto de entrenamiento o el 0.23 del conjunto de outliers. Este incremento, especialmente por ser de casos de diabéticos, es significativo, por lo que realmente lo que harían los outliers sería suavizar ese aumento, y parecer que no existe, como vemos a continuación:

In [ ]:
frontera_superior_y_outliers = pd.concat([outliers_superiores, frontera_superior])
clase_frontera_superior_y_outliers = frontera_superior_y_outliers['Outcome'].value_counts()

print("\nProporción de valores en la frontera superior junto con outliers:")
display(clase_frontera_superior_y_outliers/clase_frontera_superior_y_outliers.sum())

En la frontera, si añadimos los outliers, tenemos una pérdida de información, ya que volvemos a las proporciones iniciales de la clase. Por tanto, deberíamos eliminar los outliers de `Age`.

Ahora, vamos a hacer un análisis multivariado, en concreto vamos a analizar pares de variables predictoras visualizando la dispersión de los valores de la clase, para determinar qué variables pueden estar relacionadas o aportar información en conjunto, y también para ver qué tipo de discretización podríamos realizar.

In [ ]:
utils.plot_pairplot(data_train[["Outcome","Pregnancies","Glucose","BMI","DiabetesPedigreeFunction","Age"]],target="Outcome")

En general, no existen puntos de corte claros en prácticamente ninguna variable que dividan particiones en las que haya una mayoría clara de valores de la clase distintos. Las únicas dos variables en las que en parte de su dominio no se produce tanto solapamiento de los dos valores de la clase son `Glucose` y `BMI`, y en ambos coincide que en valores bajos (aproximadamente menor que 100 para `Glucose` y menor que 25 para `BMI`) prácticamente no hay casos con diabetes. En estos dos casos podríamos discretizar por anchura en 3 intervalos, para intentar obtener un intervalo (el primero) que comprenda esta amplia mayoría de casos que no tienen diabetes.

Para el resto de variables, puesto que posteriormente tenemos que entrenar un árbol de clasificación con y sin discretizado, podemos simplemente establecer un número habitual de intervalos, como 3 o 5, discretizando por anchura.


In [ ]:
utils.plot_conditional_barplot(data_train,"Outcome",normalize=False,marginal='box')

A pesar de que no existen discretizaciones claras como en la base de datos iris, mediante la gráfica anterior y  esta de barras condicionada podemos intuir cuales serán los atributos que jugarán un papel más importante en la clasificación.

Si seleccionamos las distribuciones de `Glucosa` o de `BMI` podemos ver como claramente para valores altos en el eje X la distribución de la variable clase cambia siendo mayor para `Diabetes`.

Por último vamos a contar el número total de outliers que vamos a eliminar:

In [ ]:
variables_con_outliers = ["BloodPressure","BMI","Age","Pregnancies"]
n_outliers,_ = utils.count_outliers(data=data_train,columns = variables_con_outliers)
print(f"Número total de outliers {n_outliers}")
print(f"Porcentaje de registros eliminados {n_outliers/data_train.shape[0]*100:.2f} %")

El número de *outliers* no supone una cantidad significativa por lo que podemos eliminarlos sin que haya demasiada perdida de información.

### 2. Preprocesamiento de datos.

Tras el análisis que hemos realizado, vamos a enumerar las transformaciones que vamos a realizar sobre los datos de entrenamiento:

- Eliminación de las variables `Insulin` y `SkinThickness` por la gran cantidad de valores desconocidos que tienen. Utilizaremos la clase 
`ColumnRemoverTransformer` que hemos creado en `utils` para ellos.
- Sustitución de los valores perdidos por el valor `np.nan` para facilitar su detección en el `Pipeline` y evitar que se incluya en cálculos como el rango intercuartil o la media. Utilizaremos la clase `Replace` creada para ese fin.
- Imputación de valores desconocidos:
	- Utilizaremos la clase `SimpleImputer` para la variable `Glucose`, puesto que la proporción de valores desconocidos es pequeña, y podemos imputar por la media directamente. Para el `Pipeline` con discretizado hemos implementado la clase `CustomSimpleImputer` que nos permite imputar las columnas que especifiquémos en los parámetros y mantener un `DataFrame` después de la transformación. Hemos utilizado este recurso ya que las transformaciones realizadas dentro de un `ColumnTransformer` se hacen por separado y luego se agrega el resultado devolviéndolo en un array de `numpy`, en este caso necesitamos una transformación secuencial ya que primero imputamos la variable `Glucose` y luego la discretizamos.
	- Para `BloodPressure` y `BMI`, al tener un número mayor de valores desconocidos, utilizaremos un árbol de regresión. Para ello utilizaremos la clase `DecisionTreeImputer` que encapsula a un árbol de regresión y toma como parámetros las columnas que se deberán tratar. Al igual que antes puesto que necesitamos una transformación secuencial que mantenga el orden de las columnas para primero imputar y luego discretizar, no podemos utilizar la clase `ColumnTransformer`.
- Eliminaremos los outliers de las variables `BloodPressure`, `Age`,`Pregnancies` y `BMI`, utilizando el criterio anterior que involucra el rango intercuartil. Hemos implementado la clase `OutliersRemovalTransformer` que extiende a `BaseSampler` de `imblearn` y elimina los  outliers en las variables predictoras especificadas actualizando también el vector de la variable clase.
- Para el segundo `pipeline`, discretizaremos las variables predictoras utilizando la clase `KBinsDiscretizer`, definiendo para todas ellas 3 intervalos uniformes (por anchura), puesto que es el tipo de discretizado que más nos convenía para las variables `Glucose` y `BMI` (para las demás variables realmente da igual, no hay puntos de corte claros para hacer intervalos más o menos puros).

In [ ]:
X_train[X_train["Glucose"]==0]

En primer lugar, preparamos una lista de transformadores sin el discretizado:

In [ ]:
transformers_sin_discretizado = []

transformers_sin_discretizado.append(utils.ColumnRemoverTransformer(["Insulin","SkinThickness"]))
transformers_sin_discretizado.append(utils.Replace(["Glucose","BloodPressure","BMI"], 0, np.nan))
variables_con_outliers = ["BloodPressure", "Age","Pregnancies","BMI"]
transformers_sin_discretizado.append(utils.OutliersRemovalTransformer(variables_con_outliers))
transformers_sin_discretizado.append(utils.DecisionTreeImputer(seed=seed,target="BMI", columns=["BMI","Glucose","Age","Pregnancies","DiabetesPedigreeFunction"]))
transformers_sin_discretizado.append(utils.DecisionTreeImputer(seed=seed,target="BloodPressure", columns=["BloodPressure","Glucose","Age","Pregnancies","DiabetesPedigreeFunction"]))
transformers_sin_discretizado.append(ColumnTransformer(transformers = [("Imputación de Glucosa",SimpleImputer(),["Glucose"])],remainder="passthrough"))

A pesar de que en este caso no existe ningún discretizado claro, puesto que las instrucciones de la práctica especifican que se deben hacer ambos, haremos un discretizado general para todas las variables. Por lo tanto creamos otra lista de transformadores, esta vez añadiendo el discretizado general en tres intervalos para ver el resultado y poder realizar alguna comparación:

In [ ]:
transformers_con_discretizado = []

transformers_con_discretizado.append(utils.ColumnRemoverTransformer(["Insulin","SkinThickness"]))
transformers_con_discretizado.append(utils.Replace(["Glucose","BloodPressure","BMI"], 0, np.nan))
transformers_con_discretizado.append(utils.OutliersRemovalTransformer(variables_con_outliers))
transformers_con_discretizado.append(utils.DecisionTreeImputer(seed=seed,target="BMI", columns=["BMI","Glucose","Age","Pregnancies","DiabetesPedigreeFunction"]))
transformers_con_discretizado.append(utils.DecisionTreeImputer(seed=seed,target="BloodPressure", columns=["BloodPressure","Glucose","Age","Pregnancies","DiabetesPedigreeFunction"]))
transformers_con_discretizado.append(utils.CustomSimpleImputer(["Glucose"])) #Necesario para poder hacer el discretizado después, ya que va a devolver un pd.DataFrame en vez de un np.array, y así podremos indexar las columnas por su nombre
transformers_con_discretizado.append(ColumnTransformer(transformers = [("Discretizado",KBinsDiscretizer(n_bins=3,strategy="uniform"),["BMI","BloodPressure","Glucose","Age","Pregnancies","DiabetesPedigreeFunction"])],remainder="passthrough"))

### 3. Aprendizaje y evaluación de un clasificador Zero-R.

El primer algoritmo que vamos a utilizar para aprender un clasificador es Zero-R (clase `DummyClassifier` en `sklearn`), que nos va a servir de ***baseline***, es decir, su resultado nos dará la medida mínima de precisión que todos los modelos que obtengamos deberán igualar o superar, puesto que éste se puede obtener de forma trivial estudiando directamente nuestro conjunto de datos (en este caso, predice siempre la clase mayoritaria, que es 0 o 'NB', es decir, no diabético, que supone aproximadamente el 65% de los valores de la clase en el conjunto de entrenamiento).

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

Al etiquetar todos los casos con la clase mayoritaria, en este caso `No diabético`, el número de verdaderos positivos y de falsos positivos es 0. 
Puesto que el `holdout` se realizó de forma estratificada la distribución de la variable clase se mantiene y el porcentaje de acierto es muy similar a la proporción de `No diabéticos` observada en el análisis exploratorio. 

Haciendo este tipo de clasificación obtenemos un porcentaje muy elevado de falsos negativos ya que todos los `Diabéticos` se predicen como `No díabeticos`.

### 4. Aprendizaje y evaluación de un árbol de decisión (sin y con discretización).


Ahora, vamos a utilizar un algoritmo más complejo que en principio debería aprender un modelo que mejore los resultados del anterior. En particular, utilizaremos un algoritmo para aprender árboles de clasificación, `DecisionTreeClassifier`, utilizando la entropía como criterio de selección de puntos de corte en las variables predictoras, y por simplificar y no sobreajustar a los datos una profundidad máxima de 5.

En primer lugar, vamos a trabajar sin discretizado; creamos el objeto que encapsulará el algoritmo y el modelo (árbol), y lo añadimos a la lista de transformadores, puesto que se la pasaremos al pipeline.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed, max_depth=5, criterion="entropy")
transformers_sin_discretizado.append(tree_model)

Utilizaremos el `Pipeline` de la librería `imblearn` en vez del de `sklearn` puesto que nos permite acceder a la variable objetivo y asi realizar operaciones como el borrado de filas, que utilizaremos para eliminar *outliers*, tanto en las variables predictoras como en la variable clase. Los transformadores que modifican tanto las variables predictoras como la variable clase extenderán a la clase `BaseSampler` e implementarán los métodos `fit_resample` y `_fit_resample`.

In [ ]:
pipeline_sin_discretizado = imblearn_make_pipeline(*transformers_sin_discretizado)
utils.evaluate(pipeline_sin_discretizado,
               X_train, X_test,
               y_train, y_test)

print(utils.evalute_classification_report(pipeline_sin_discretizado,
               X_train, X_test,
               y_train, y_test))

La tasa de acierto obtenida es del 72%, la mejora con respecto al `ZeroR` no es significativa si tenemos en cuenta el aumento en la complejidad del modelo.

Obviamente al no realizar una predicción estática como en el `ZeroR` donde todos los casos se diagnostican como `No diabéticos` el número de falsos negativos disminuye.


In [ ]:
utils.plot_roc_curve(pipeline_sin_discretizado,
                    X_train,y_train,
                    X_test,y_test,
                    pos_label='D')

Creamos el pipeline esta vez incluyendo los transformadores para el discretizado.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed, max_depth=5, criterion="entropy")
transformers_con_discretizado.append(tree_model)

In [ ]:
pipeline_con_discretizado = imblearn_make_pipeline(*transformers_con_discretizado)
utils.evaluate(pipeline_con_discretizado,
               X_train, X_test,
               y_train, y_test)

print(utils.evalute_classification_report(pipeline_con_discretizado,
               X_train, X_test,
               y_train, y_test))

La precisión a aumentado pasando de 72% a 77% utilizando el discretizado, el número de falsos positivos ha dismunido en 16 casos sin embargo el número de falsos negativos ha aumentado en 3 casos. 

Dada la semantica del problema la mejora es importante ya que un falso positivo puede tener consecuencias graves ya que estaríamos administrando insulina a personas sanas.

In [ ]:
utils.plot_roc_curve(pipeline_con_discretizado,
                    X_train,y_train,
                    X_test,y_test,
                    pos_label='D')

Los datos observados en la matriz de confusión se pueden visualizar en la curva ROC, donde vemos que el área ha aumentado con respecto a la curva ROC y las coordenadas han cambiado. El valor en la tasa de verdaderos positivos ha disminuido lo que indica que ha aumentado el de falsos negativos.

# Conjunto de datos II: Breast Cancer (Wisconsin)

Procederemos a realizar el estudio de manera análoga, empezando por la carga de los datos y la partición en conjunto de _test_ y de _entrenamiento_

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data_breast_cancer = utils.load_data(filepath, index, target)
data_breast_cancer.drop(columns=['Unnamed: 32'],inplace=True)
(X, y) = utils.divide_dataset(data_breast_cancer, target=target)

In [ ]:
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)


data_train = utils.join_dataset(X_train,y_train)

Comprobamos que los datos se han cargado correctamente. Con un muestreo aleatorio

In [ ]:
data_train.sample(5,random_state=seed)

### 1. Análisis exploratorio de datos mediante gráficas y estadísticos.

En primer lugar, vamos a ver qué tipo de datos tenemos tanto en las variables predictoras como en la objetivo:

In [ ]:
data_train.info(memory_usage=False)

Todas las variables predictoras son numéricas, mientras que la variable objetivo es categórica, por lo que estamos ante un problema de clasificación. Si lo comparamos con el conjunto de datos de diabetes, este conjunto de entrenamiento es más reducido en cuanto a registros pero vamos extenso en cuanto a número de variables. Veamos ahora más datos sobre cada variable:

In [ ]:
display(data_train.describe(include="number"))
display(data_train.describe(include="category"))

Vamos a estudiar la distribución de las variables numéricas

In [ ]:
utils.plot_histogram(data_train)

En general, la distribución que siguen todas las variables es normal, aunque hay ciertas particularidades como `area_se` que tienen más dispersión en su dominio y un grupo de datos muy concentrado, por tanto la escala no deja ver esa distribución normal o parecida a la normal.

El dataset está formado por 30 variables predictoras numéricas y una variable clase. Las variables predictoras son en realidad 10, donde para cada una de ellas se nos proporciona `mean` (media), `se` (error estándar) y `worst` (el peor valor siendo éste la media de los tres valores más grandes, según la documentación). La variable objetivo tiene sólamente dos clases, que podemos ver a continuación:

In [ ]:
utils.plot_barplot(data_train)

En este caso la base de datos tampoco está balanceada. La clase mayoritaria es `B` con un 62.8% lo cual indica que el tumor es benigno, mientras que la clase `M` supone un 37.2%, indicando que el tumor es maligno.

En primer lugar, dado que por su nombre hay varias variables predictoras que podrían estar muy relacionadas, por ejemplo las referentes a `radius`, `area` y `perimeter`, y el número de variables es muy alto para visualizar todas las relaciones entre ellas, vamos a realizar primero un análisis multivariado, por lo que dividiremos las gráficas en 2 grupos de variables:
- [`radius_mean`,`area_mean`,`perimeter_mean`,`smoothness_mean`,`compactness_mean`], puesto que, por razones obvias, las tres primeras están relacionadas y, en lo que respecta a `smoothness_mean` y a `compactness_mean`, la primera según la documentación está relacionada con la variación de la longitud del radio del núcleo celular y la segunda se calcula a partir del perímetro y del área.
- [`radius_mean`,`symmetry_mean`,`fractal_dimension_mean`,`texture_mean`,`concave points_mean`,`concavity_mean`], ya que en principio son variables que por su descripción no parecen tener una relación tan directa. Además, añadimos `radius_mean` como "representante" del grupo anterior y así ver la correlación entre grupos sin tener que crear todos los pares.

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.loc[:,["radius_mean","area_mean","perimeter_mean","smoothness_mean","compactness_mean"]],data_train[target]),target=target)

Como podemos ver, hay una relación directa entre `radius_mean`, `area_mean` y `perimeter_mean`, por lo que sería interesante mantener sólo una de estas variables, por ejemplo `radius_mean`, y desechar las otras dos para intentar hacer más simple el entrenamiento en términos de coste de computación (no en términos de la complejidad del entrenamiento en sí, en el caso de entrenar árboles de clasificación daría igual, porque no se agregan resultados a diferencia de otros algoritmos como Naive Bayes), ya que realmente no aportan más información que la primera y supondría analizar menos puntos de corte.

Además, otro aspecto que podemos ver es que en general podríamos discretizar las variables en dos intervalos por anchura, especialmente si eliminamos outliers previamente. Quizás en el caso de `smoothness_mean` no está tan claro este discretizado.

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.loc[:,["radius_mean","symmetry_mean","fractal_dimension_mean","texture_mean","concave points_mean","concavity_mean"]],data_train[target]),target=target)

De este grupo de variables podemos extraer que hay dos que parecen tener cierta correlación positiva con `radius_mean`, que son `concave points_mean` y `concavity_mean`, que a su vez es normal puesto que entre ellas parece haber bastante correlación también (podemos ver que las columnas de ambas variables son muy parecidas, más adelante vamos a determinar cuál nos conviene mantener). Por otro lado, también parece que `fractal_dimension_mean` decrece un poco al incrementar `radius_mean`.

Para `symmetry_mean`, `fractal_dimension_mean` y `texture_mean` no parece haber un punto de corte claro para discretizar, aunque en `concave points_mean` y `concavity_mean` parece que podemos hacer dos grupos.

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.loc[:,["radius_se","area_se","perimeter_se","smoothness_se","compactness_se"]],data_train[target]),target=target)

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.loc[:,["radius_se","symmetry_se","fractal_dimension_se","texture_se","concave points_se","concavity_se"]],data_train[target]),target=target)

En cuanto a las variables `_se`, que se refieren al error estándar obtenido para cada variable predictora, en general parece haber una gran concentración de valores en la parte inferior-media del dominio observado en cada variable, mientras que hay una gran dispersión de valores en la parte superior (en algunas variables también por el número de valores que serán en su mayoría outliers, pero en general por distancia entre estos y aquel cúmulo de valores), por lo que más adelante analizaremos si nos conviene utilizar estas variables en el entrenamiento en el caso de que un gran número de instancias fueran desechadas al eliminar outliers. Aun así, hay alguna variable como `concave points_se` y `texture_se` que no tienen tantos outliers y al estudiar combinaciones de puntos de corte en ambas puede salir particiones aprovechables (por ejemplo, cuando `concave points_se` < 0.013 aproximadamente y `texture_se` > 1.5 aprox., todos o casi todos los casos son tumores benignos).

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.loc[:,["radius_worst","area_worst","perimeter_worst","smoothness_worst","compactness_worst"]],data_train[target]),target=target)

In [ ]:
utils.plot_pairplot(utils.join_dataset(data_train.loc[:,["radius_worst","symmetry_worst","fractal_dimension_worst","texture_worst","concave points_worst","concavity_worst"]],data_train[target]),target=target)

En el caso de las variables predictoras `_worst`, podemos ver que las gráficas son muy parecidas en forma a aquellas que obtuvimos con las variables `_mean`. Esto es así porque, según la descripción de aquellas, son obtenidas como la media de los tres casos más grandes en la imagen (aunque la forma sea parecida, también podemos ver que rango de valores del dominio es precisamente algo mayor que los correspondientes de las variables `_mean`), por lo que probablemente también nos interese incluirlo en el entrenamiento puesto que puede ser determinante para detectar un caso maligno especialmente.

Veamos ahora la matriz de correlación de variables para determinar finalmente qué variables seleccionar y cuáles no aportarían más información o al menos convendría quitarlas para reducir el tiempo de entrenamiento.

In [ ]:
utils.plot_correlationmatrix(data_train)

Como podemos ver, en primer lugar, el patrón presente en las variables `_mean` se repita prácticamente de forma idéntica en las variables `_worst`, aunque parece haber mas correlación en general con el resto de variables. Por otro lado, confirmamos la obvia relación entre `radius_mean`, `perimeter_mean` y `area_mean`, por lo que mantendremos sólamente la primera. En cuanto a `concave points_mean` y `concavity_mean`, confirmamos la gran correlación que tienen, y ahora tendremos que analizar cuál de ellas mantener.

In [ ]:
utils.plot_conditional_barplot(data_train, target, normalize=True, marginal='box')

Este gráfica condicional contiene mucha información que utilizaremos principalmente para determinar el tipo de discretizado y/o el número de intervalos. Puesto que los rangos son distintos y que nos interesa evaluar la distribución las hemos normalizado para poder visualizarlas mejor. Puesto que hemos normalizado utilizando la fórmula $\dfrac{x-\mu}{\sigma}$, sabemos que el valor 0 se corresponde con la media de la variable.

-  En general observamos que una discretiación en 2 intervalos parece adecuada para la mayoría de variables. Esta gráfica muestra las variables condicionadas a la variable clase y por lo tanto los outliers observados no se traducen directamente en outliers de la variable (habrá que confirmar los *outliers* existentes en la variable sin condicionar). 

- En el apartado anterior observamos que `concave points_mean` y `concavity_mean` mostraban distribuciones y tendencias muy similares y que por lo tanto al igual que con el radio podríamos seleccionar una de las dos. En esta gráfica observamos que ambas tienen un número de outliers parecido pero `concavity_mean` tiene mayor dispersión. Para la variable `concave points_mean` distinguimos dos clusters, el primero alrededor de -0.75 y el segundo cerca de 1, por lo tanto una discretización con en 2 grupos mediante`k-means` parece la más indicada. Por lo tanto nos quedaremos con `concave points_mean` y eliminaremos `concavity_mean`en el `Pipeline`


- En esta gráfica, a diferencia de la anterior, observamos que aunque la variable `area_se` tiene los valores muy concentrados también tiene bastante poder predictivo ya que podemos distinguir claramente dos centros alrededor de los cuales se concentran los registros de cada clase.  Aunque como ya mencionamos con anterioridad el perímetro y el radio están directamente relacionados con el area, para este caso particular vemos que la concentración de los registros y la división es mucho más clara entre otras cosas debido a que hay menos *outliers* para la variable condicionada que introducen ruido.El punto de corte en este caso parece estar también cerca de la media.

- Observamos también que a parte de `area_se` y sus equivalentes, de forma univariada ningún otro atributo de tipo `_se` divide el conjunto de datos de esta manera. 

En general tras realizar el análisis para las variables intuimos que hay un número significativo de *outliers*. A continuación vamos a contar el número de registros que estaríamos eliminando, ya que teniendo un conjunto de entrenamiento con tan solo 398 registros no podemos permitirnos eliminar más registros de los necesarios. Con lo observado hasta el momento no podemos saber si estos *outliers* pertenecen todos a un grupo de registros aislado o si están repartidos por el conjunto de datos. 

Hemos de tener en cuenta que disponemos de un número elevado de variables (con redundancia de información) y un número reducido de registros por lo tanto puede que sea preferible descartar algunas variables con muchos *outliers*.

In [ ]:
utils.plot_boxplot(data_train)

In [ ]:
data_train[data_train["area_se"]>94.44]["diagnosis"].value_counts()

In [ ]:
data_train[data_train["area_se"]>data_train["area_se"].mean()]["diagnosis"].value_counts()

La variable `area_se` tiene muchos outliers, sin embargo no vamos a eliminarlos puesto que todos pertenecen a la misma clase siendo esta además la clase positiva, realmente no se estaría sobreajustando ya que el clasificador no iría a buscar ningún valor extremo para clasificarlo correctamente. Ningún discretizador nos daría la discretización que buscamos, por lo tanto implementaremos uno para que cree dos intervalos tomando como punto de corte la media de la variable.

Vamos ahora a contar el número total de outliers que estaríamos eliminando una vez seleccionadas las variables

In [ ]:
variables_seleccionadas= [
  'compactness_worst',
 'concave points_worst',
 'fractal_dimension_worst',
 'radius_worst',
 'smoothness_worst',
 'symmetry_worst',
 'texture_worst', 
 'compactness_mean',
 'concave points_mean',
 'fractal_dimension_mean',
 'radius_mean',
 'smoothness_mean',
 'symmetry_mean',
 'texture_mean',
 'texture_se',
 'concave points_se',
 'area_se'
 ]
 
outliers_total,outliers = utils.count_outliers(data_train,columns = set(variables_seleccionadas)-set(["area_se"]))
print(f"Outliers totales: {outliers_total}" )
outliers.value_counts(target,normalize=True)

Como podemos ver si incluímos todas las seleccionadas variables excepto area el número total de *outliers* representa más del 18% del total, incluso seleccionando solo un subconjunto de variables el número de outliers sigue siendo elevado. Estos datos indican que la cardinalidad de la intersección entre los *outliers* observados en el punto anterior es muy baja. 

Además si analizamos el porcentaje de `Maligno` y `Benigno` vemos que se ha invertido respecto a la proporción del conjunto de entrenamiento. Pues que la base de datos no está balanceada podemos considerar que esta información es valiosa y quitarla sería contraproducente ya que estaríamos descompensando todavía más la distribución.

En el [artículo](https://www.researchgate.net/publication/2512520_Nuclear_Feature_Extraction_For_Breast_Tumor_Diagnosis) que describe el estudio a partir del cual se obtuvieron los datos. Se especifica que: "Todos los atributos han sido modelados para que cuanto mayor sea el valor mayor será la probabilidad de que el tumor sea `Maligno`".


Puesto que no vamos a quitar *outliers* una discretización por anchura generaría intervalos válidos. Del mismo modo no parece que una discretización por frecuencia sea válida, así que ya que para la mayoría de variables podíamos ver dos `clusters` más o menos disjuntos, vamos a realizar una discretización algo más informada utilizando el `k-means`.

Como en el apartado anterior crearemos dos `Pipelines` uno con discretizado y otro sin discretizado. Para el segundo puede ser interesante analizar si un redondeando los datos podemos reducir el sobreajuste

In [ ]:
for variable in variables_seleccionadas:
  utils.round_values_analysis(data_train,target,variable,2)
  print()

En principio vemos que o bien el redondeo no cambia el número de valores únicos en la variable y por lo tanto no acarrea una pérdida de información o bien las agrupaciones resultantes del redondeo son demasiado grande y hacen que se pierda demasiada información. Por ejemplo para la variable `fractal_dimension_mean` con un redondeo de 2 decimales obtenemos solo 6 valores únicos por lo que el número de casos cuyo valor de clase difiere de la mayoritaria para ese redondeo aumenta mucho.

Por lo tanto no redondearemos ninguna variable.

### 2. Preprocesamiento de datos.

Creamos dos variables para las columnas seleccionadas y las columnas a eliminar

In [ ]:
variables_seleccionadas= [
  'compactness_worst',
 'concave points_worst',
 'fractal_dimension_worst',
 'radius_worst',
 'smoothness_worst',
 'symmetry_worst',
 'texture_worst', 
 'compactness_mean',
 'concave points_mean',
 'fractal_dimension_mean',
 'radius_mean',
 'smoothness_mean',
 'symmetry_mean',
 'texture_mean',
 'texture_se',
 'concave points_se',
 "area_se"
 ]
variables_a_eliminar = set(X_train.columns)-set(variables_seleccionadas)

En primer lugar definimos una lista de transformadores con discretizado

In [ ]:
transformers_con_discretizado = []

transformers_con_discretizado.append(utils.ColumnRemoverTransformer(columns_to_drop=variables_a_eliminar))
transformers_con_discretizado.append(utils.MeanDiscretizer(["area_se"]))
transformers_con_discretizado.append(
    ColumnTransformer(transformers = [
                            ("K-MEANS DISC.",KBinsDiscretizer(n_bins=2,strategy="kmeans"),list(set(variables_seleccionadas)-set(["area_se"])))
                            ]
                             ,remainder="passthrough"))

Repetimos el proceso pero para una lista de transformadores esta vez sin discretizado

In [ ]:
transformers_sin_discretizado = []
transformers_sin_discretizado.append(utils.ColumnRemoverTransformer(columns_to_drop=variables_a_eliminar))

### 3. Aprendizaje y evaluación de un clasificador Zero-R.

Al igual que en la primera parte de la práctica utilizaremos como *_baseline_* el algoritmo de clasificación ZeroR que hará una clasificación constante asociando en este caso cualquier registro a predecir a la clase mayoritaria `Benigno`.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

utils.evaluate(zero_r_model,
               X_train, X_test,
               y_train, y_test)

Como podemos ver, el *baseline* en este caso es aproximadamente 0.626, que es la proporción de casos de tumor benigno en el conjunto de entrenamiento, por lo que cualquier otro modelo que entrenemos tendrá que igualar o superar este valor de precisión.

### 4. Aprendizaje y evaluación de un árbol de decisión (sin y con discretización).


Creamos dos `Pipelines`, uno con discretizado y otro sin discretizado

In [ ]:
transformers_sin_discretizado.append(DecisionTreeClassifier(criterion="entropy",random_state=seed))
transformers_con_discretizado.append(DecisionTreeClassifier(criterion="entropy",random_state=seed))

Vamos a evaluar en primer lugar el `Pipeline` sin discretizado

In [ ]:
pipeline_sin_discretizado = imblearn_make_pipeline(*transformers_sin_discretizado)
utils.evaluate(pipeline_sin_discretizado,
               X_train, X_test,
               y_train, y_test)

print(utils.evalute_classification_report(pipeline_sin_discretizado,
               X_train, X_test,
               y_train, y_test))

La tasa de acierto obtenida es del 92%, la mejora con respecto al `ZeroR` es significativa. Aunque dada la semántica del problema lo que realmente nos importa en este caso son los `falsos negativos` que podemos ver en el resumen que aparece encima de la matriz de confusión. El modelo obtiene un `recall` de 0.81 para `Maligno`, lo cual se traduce en que se estarían diagnosticando como `Benignos` tumores que son en realidad malignos.


In [ ]:
utils.plot_roc_curve(pipeline_sin_discretizado,
                    X_train,y_train,
                    X_test,y_test,
                    pos_label='M')

El modelo es mucho más preciso que los estimadores triviales y esto se traduce en que el área bajo la curva es cercana a 1. El punto débil de este modelo frente a uno perfecto es el `recall` que se traduce gráficamente en el espacio que falta para completar el área.

Realizamos el mismo proceso ahora para el `Pipeline` con discretizado

In [ ]:
pipeline_con_discretizado = imblearn_make_pipeline(*transformers_con_discretizado)
utils.evaluate(pipeline_con_discretizado,
               X_train, X_test,
               y_train, y_test)

print(utils.evalute_classification_report(pipeline_con_discretizado,
               X_train, X_test,
               y_train, y_test))

La precisión a disminuido frente al `Pipeline` sin discretizado pasando de 92% a 94%. Además el recall ha aumentado en 7 puntos de porcentaje, donde sólo 8 casos positivos se han predicho como negativos.

In [ ]:
utils.plot_roc_curve(pipeline_con_discretizado,
                    X_train,y_train,
                    X_test,y_test,
                    pos_label='M')

Si evaluamos los modelos por el `AUC` obtenido vemos que ha mejorado. Puesto que no sólo ha mejorado el `AUC` sino también el *recall* este último modelo es preferible.